# Branch-and-Bound: The Travelling Salesperson

In homework 8, we implemented The Traveling Salesperson problem using dynamic programming. In this homework, we will use the branch and bound technique to solve The Traveling Salesperson problem!

### If you're using Datahub:
* Run the cell below **and restart the kernel if needed**

### If you're running locally:
You'll need to perform some extra setup.
#### First-time setup
* Install Anaconda following the instructions here: https://www.anaconda.com/products/distribution 
* Create a conda environment: `conda create -n cs170 python=3.8`
* Activate the environment: `conda activate cs170`
    * See for more details on creating conda environments https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html
* Install pip: `conda install pip`
* Install jupyter: `conda install jupyter`

#### Every time you want to work
* Make sure you've activated the conda environment: `conda activate cs170`
* Launch jupyter: `jupyter notebook` or `jupyter lab` 
* Run the cell below **and restart the kernel if needed**

In [ ]:
# Install dependencies
!pip install -r requirements.txt --quiet

In [ ]:
import otter

assert (otter.__version__ >= "4.4.1"), "Please reinstall the requirements and restart your kernel."

grader = otter.Notebook("hw13.ipynb")
import numpy as np
import networkx as nx
from networkx import Graph, draw
import math
import numpy.random as random
import heapq
import string
import pylev
import tqdm
from time import time
import pickle 

from autograder_utils import validate_tour, handle_timeout, profile

test_data = pickle.load(open("public_data.pkl", "rb"))

rng_seed = 0

## Q1: Traveling Salesperson

In this problem, we will implement the Traveling Salesperson Problem (TSP) using branch and bound. While dynamic programming guarantees a solution in exponential time, branch and bound can often be significantly faster by pruning suboptimal paths.

For a refresher on the TSP and the branch and bound approach, see Lecture 24 or https://people.eecs.berkeley.edu/~vazirani/algorithms/chap9.pdf#page=5.

### Graph helpers
Like the last homeworks, we use a weighted adjacency list to represent the graph. We'll use a similar format as before, except `graph[u]` is now a hashmap instead of a list of pairs. 
 **For this assignment, graphs are undirected**, so if there is an (undirected) edge between nodes `u` and `v` with weight `w`, then `graph[u]` contains key `v` with value `w` and `graph[v]` contains key `u` with value `w`.

We provide the following code to help you test your implementation.

In [ ]:
def generate_adj_list(n, edge_list: list[tuple[int]]) -> list[dict[int, int]]:
    """
    args:
        n:int = number of nodes in the graph. The nodes are labelled with integers 0 through n-1
        edge_list:list[tuple[int,int,int]] = edge list where each tuple (u,v,w) represents the 
        undirected and weighted edge (u,v,w) in the graph
    return:
        A list[dict[int, int]] representing the adjacency list 
    """
    adj_list = [{} for _ in range(n)] 
    for u, v, w in sorted(edge_list):        
        adj_list[u][v] = w
        # undirected edges
        adj_list[v][u] = w
    return adj_list

def draw_graph(adj_list: list[dict[int, int]]):
    """Utility method for visualizing graphs

    args:
        adj_list: list[dict[int, int]] = adjacency list representation of the graph generated by generate_adj_list
    """
    G = Graph()
    for u in range(len(adj_list)):
        for v, w in adj_list[u].items():
            G.add_edge(u, v, weight=w)
    draw(G, with_labels=True)

When computing the lower bound, you will need to find the cost of the Minimum Spanning Tree (MST) over the remaining untraversed vertices. The following code implements Prim's algorithm to compute the MST cost from an adjacency list.

Note: __The input adjacency list includes al vertices from the original TSP graph.__ This means that if you do not want to include a vertex in the output MST, remove all the edges from that vertex! This is just for simplicity in the staff solution but you are more than welcome to change any part of the code.

In [ ]:
def mst(adj_list):
    num_vertices = len(adj_list)
    visited = [False] * num_vertices
    mst_cost = 0
    
    start_vertex = 0
    for i in range(num_vertices):
        if adj_list[i]:  # If this vertex has any edges
            start_vertex = i
            break
    
    # start from vertex 0
    min_heap = [(0, start_vertex, -1)]  # (weight, vertex, parent)
    
    while min_heap:
        w, u, parent = heapq.heappop(min_heap)
        
        if visited[u]:
            continue
            
        visited[u] = True
        
        if parent != -1:  # Not the starting vertex
            mst_cost += w
        
        for v, w in adj_list[u].items():
            if not visited[v]:
                heapq.heappush(min_heap, (w, v, u))
#     print(mst_cost)
    
    return mst_cost

### A. Find the lower bound
In branch and bound, we use a lower bound to decide whether a subproblem is worth exploring. There are various ways to compute this bound, but for this homework, we'll follow the approach from lecture 24 or the [textbook](https://people.eecs.berkeley.edu/~vazirani/algorithms/chap9.pdf#page=6).

As a reminder, the lower bound for a subproblem $T[S, j]$ is defined as:

$$\text{lower\_bound}(T[S, j]) = \text{MST}(V \setminus S) + \min_{x \in V \setminus S} d_{1x} + \min_{y \in V \setminus S} d_{jy} + T[S, j]$$

In [ ]:
def lower_bound(adj_list, path, path_cost, remaining_vertices):
    """ Returns the lower bound of the current subproblem.

    Args:
        adj_list (list[dict[int, int]]): Weighted undirected graph represented as an adjacency list.
        path (list[int]): A list of vertices that have been traversed.
        path_cost (int): The cost of traversed path.
        remaining_vertices(Set[int]): A set of untraversed vertices.

    Returns:
        int: The lower bound of current subproblem.
    """
    ...

In [ ]:
grader.check("q1")

### B. Implement TSP using branch and bound
Now, implement The Traveling Salesperson problem using branch and bound. You may follow the pseudocode found in Lecture 24 or the [textbook](https://people.eecs.berkeley.edu/~vazirani/algorithms/chap9.pdf#page=5).

As with previous problems, your function should return the actual tour, not the cost of the tour.
__Do not add the starting vertex to the end of your output. The length of your output tour should equal the number of cities (vertices).__

For example, if there are 5 cities, a valid output is: $[0, 1, 2, 3, 4]$.

Your implementation will be tested for correctness and speed. For the first set of tests, at least 15 / 20 speed tests must pass. For the second set of tests, at least 22 / 30 speed tests must pass.

In [ ]:
def tsp(adj_list):
    """Compute the exact solution to the TSP using dynamic programming and returns the optimal path.

    Args:
        adj_list: Weighted undirected graph represented as an adjacency list. 

    Returns:
        List[int]: A list of city indices representing the optimal path. Do not include the starting vertex in the end.
    """
    ...

In [ ]:
grader.check("q2")

### Debugging
The following example is from the [textbook](https://people.eecs.berkeley.edu/~vazirani/algorithms/chap9.pdf#page=7). Feel free to add whatever print statements or assertions you'd like when debugging.

In [ ]:
# Example from textbook
n = 8
edge_list = [
    (0, 1, 2),
    (0, 5, 1),
    (0, 7, 1),
    (1, 2, 1),
    (1, 4, 1),
    (2, 7, 5),
    (2, 3, 1),
    (3, 4, 2),
    (3, 6, 1),
    (4, 5, 1),
    (5, 6, 2),
    (6, 7, 1),
]

# Generate adjacency list
adj_list = generate_adj_list(n, edge_list)

# Expected optimal tour and cost for this example
expected_tour = [0, 5, 4, 1, 2, 3, 6, 7]
expected_cost = 8

tsp(adj_list)

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit.

In [ ]:
grader.export(pdf=False, force_save=True, run_tests=True)